In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen
from collections import deque


import seaborn as sns

%matplotlib inline

In [2]:
df = pd.read_csv('../input/2012_qb_weeks1_8.csv')
df1=pd.read_csv('../input/2012_qb_weeks_9_16.csv')
df3 = pd.read_csv('../input/2012_qb_week17.csv')

In [3]:
raw_qb_2012 = pd.concat([df,df1], axis=0)
raw_qb_2012 = pd.concat([raw_qb_2012, df3], axis=0)
raw_qb_2012.drop('Rk', axis =1, inplace = True)
raw_qb_2012.shape

(629, 18)

In [4]:
raw_qb_2012.head()

,Player,Pos,Week,Team,Opp,Comp,Att,Pct,Yds,Yds/Att,TD,Int,QB Rating,Att.1,Yds.1,Yds/Att.1,TD.1,Fantasy Points
0,Matt Ryan,QB,1,ATL,KC,23,31,74,299,10,3,0,136,3,25,8,1,32.46
1,Robert Griffin III,QB,1,WAS,NO,19,26,73,320,12,2,0,140,9,42,5,0,25.00
2,Tony Romo,QB,1,DAL,NYG,22,29,76,307,11,3,1,130,5,12,2,0,23.48
3,Aaron Rodgers,QB,1,GB,SF,30,44,68,303,7,2,1,93,5,27,5,0,22.82
4,Drew Brees,QB,1,NO,WAS,24,52,46,339,7,3,2,71,0,0,0,0,21.56


In [5]:
raw_qb_2012 = raw_qb_2012.rename(columns={'Att.1':'rush_att',
                       'Yds.1':'rush_yds',
                       'Yds/Att.1':'rush_yd/att',
                       'TD.1':'rush_td'})

In [6]:
team_stats = raw_qb_2012.groupby(['Player','Week','Opp']).sum()

In [7]:
team_stats.head(), team_stats.shape

(                        Comp  Att  Pct  Yds  Yds/Att  TD  Int  QB Rating  \
 Player        Week Opp                                                     
 Aaron Rodgers 1    SF     30   44   68  303        7   2    1         93   
               2    CHI    22   32   69  219        7   1    1         85   
               3    SEA    26   39   67  223        6   0    0         81   
               4    NO     31   41   76  319        8   4    1        120   
               5    IND    20   32   63  235        7   3    1        103   
 
                         rush_att  rush_yds  rush_yd/att  rush_td  \
 Player        Week Opp                                             
 Aaron Rodgers 1    SF          5        27            5        0   
               2    CHI         3        -6           -2        0   
               3    SEA         2        17            9        0   
               4    NO          5        13            3        0   
               5    IND         5        57  

In [11]:
y=weekly_stats['Fantasy Points']

In [8]:
# Define a dataframe to contain the current year 3 game moving averages of all 2012 QBs
def create_recent_dataframe(df):
    columns = []
    for col in df.columns:
        columns.append('{}_ma'.format(col.lower()))
    new_df = pd.DataFrame(columns=columns, index =df.index)
    return new_df

In [9]:
# Create recent and current dataframes for all 2012 QBs
qb_recent_2012= create_recent_dataframe(team_stats)

In [55]:
#Populate recent dataframe with 3 game moving average for all qb's in 2012 season
# DO NOT CHANGE !!!!!!!!
for col in team_stats:  # Iterate through every attribute in the team_stats dataframe
    ma3 = []  # Create an empty list to hold the season average of the current attribute
    for player in team_stats.index.levels[0]:  # Iterate through every player in the team_stats df
        total = deque([], maxlen = 3) # INstantiate a deque container with a max length of 3 to hold the three most current games for player
        count = 0 # Counter to track week number 
        for week in team_stats.loc[player].index: # Iterate through every week for relevant player d
            if count < 3: # check count value to start 3 game moving average
                ma3.append(0)  # week 1 through 3 have 3 game movong average of 0
                total.appendleft(team_stats.loc[player].loc[week][col])  # Add the value current player's attribute total from the left
                count += 1  # Increase count
            else:  # Once 3 game moving average is avaliable
                ma3.append(np.mean(total))  # Add the average of the 3 games held in totals 3 game moivng average list
                total.appendleft(team_stats.loc[player].loc[week][col]) # Replace 3 set block with a first in first out process
    qb_recent_2012['{}_ma'.format(col.lower())] = ma3  # Populate databse with 3 game moving average list

In [61]:
qb_recent_2012.shape

(629, 13)

In [60]:
qb_recent_2012[0:100]

comp_ma     att_ma     pct_ma      yds_ma  \
Player             Week Opp                                                
Aaron Rodgers      1    SF    0.000000   0.000000   0.000000    0.000000   
                   2    CHI   0.000000   0.000000   0.000000    0.000000   
                   3    SEA   0.000000   0.000000   0.000000    0.000000   
                   4    NO   26.000000  38.333333  68.000000  248.333333   
                   5    IND  26.333333  37.333333  70.666667  253.666667   
                   6    HOU  25.666667  37.333333  68.666667  259.000000   
                   7    LAR  25.000000  36.666667  68.000000  297.333333   
                   8    JAX  24.666667  35.333333  69.666667  305.000000   
                   9    ARI  25.333333  36.333333  69.666667  288.666667   
                   11   DET  22.000000  34.000000  63.666667  248.666667   
                   12   NYG  18.333333  30.666667  60.000000  213.333333   
                   13   MIN  15.666667  27.333333  57.666667  224.333333   
                   14   DET  20.000000  29.000000  67.666667  247.000000   
                   15   CHI  18.333333  28.000000  63.666667  226.000000   
                   16   TEN  21.333333  31.666667  66.333333  250.000000   
                   17   MIN  21.333333  32.666667  64.333333  268.666667   
Alex Smith         1    GB    0.000000   0.000000   0.000000    0.000000   
                   2    DET   0.000000   0.000000   0.000000    0.000000   
                   3    MIN   0.000000   0.000000   0.000000    0.000000   
                   4    NYJ  21.333333  30.666667  70.333333  213.666667   
                   5    BUF  18.666667  29.000000  63.666667  191.000000   
                   6    NYG  18.000000  26.666667  67.000000  216.666667   
                   7    SEA  16.333333  25.000000  65.000000  215.333333   
                   8    ARI  17.000000  25.666667  66.333333  214.333333   
                   10   LAR  17.000000  24.000000  73.000000  190.666667   
                   11   CHI  13.000000  16.666667  81.333333  148.000000   
                   12   NO    8.333333   9.000000  61.000000  101.333333   
                   13   LAR   2.333333   2.666667  29.333333   24.000000   
                   14   MIA   0.000000   0.000000   0.000000    0.000000   
                   15   NE    0.000000   0.000000   0.000000    0.000000   
...                                ...        ...        ...         ...   
Ben Roethlisberger 8    WAS  24.333333  38.333333  63.666667  287.000000   
                   9    NYG  25.000000  36.666667  68.666667  287.666667   
                   10   KC   24.000000  33.333333  72.000000  238.666667   
                   14   LAC  18.000000  27.000000  64.333333  174.000000   
                   15   DAL  17.333333  30.000000  57.333333  195.000000   
                   16   CIN  18.333333  33.333333  54.000000  236.000000   
                   17   CLE  20.000000  36.666667  54.000000  281.333333   
Blaine Gabbert     1    MIN   0.000000   0.000000   0.000000    0.000000   
                   2    HOU   0.000000   0.000000   0.000000    0.000000   
                   3    IND   0.000000   0.000000   0.000000    0.000000   
                   4    CIN  13.333333  26.333333  48.000000  156.000000   
                   5    CHI  13.333333  24.666667  51.000000  131.333333   
                   7    OAK  16.666667  29.333333  56.000000  161.000000   
                   8    GB   16.000000  26.333333  62.333333  146.000000   
                   9    DET  17.333333  31.333333  58.000000  185.000000   
                   10   IND  20.666667  33.000000  64.333333  211.000000   
                   11   HOU  24.000000  39.333333  61.333333  244.000000   
Brady Quinn        5    BAL   0.000000   0.000000   0.000000    0.000000   
                   6    TB    0.000000   0.000000   0.000000    0.000000   
                   8    OAK   0.000000   0.000000   0.000000    0.000000   
   

In [63]:
qb_recent_2012.head()

comp_ma     att_ma     pct_ma      yds_ma  \
Player        Week Opp                                                
Aaron Rodgers 1    SF    0.000000   0.000000   0.000000    0.000000   
              2    CHI   0.000000   0.000000   0.000000    0.000000   
              3    SEA   0.000000   0.000000   0.000000    0.000000   
              4    NO   26.000000  38.333333  68.000000  248.333333   
              5    IND  26.333333  37.333333  70.666667  253.666667   

                        yds/att_ma     td_ma    int_ma  qb rating_ma  \
Player        Week Opp                                                 
Aaron Rodgers 1    SF     0.000000  0.000000  0.000000      0.000000   
              2    CHI    0.000000  0.000000  0.000000      0.000000   
              3    SEA    0.000000  0.000000  0.000000      0.000000   
              4    NO     6.666667  1.000000  0.666667     86.333333   
              5    IND    7.000000  1.666667  0.666667     95.333333   

                        rush_att_ma  rush_yds_ma  rush_yd/att_ma  rush_td_ma  \
Player        Week Opp                                                         
Aaron Rodgers 1    SF      0.000000     0.000000        0.000000         0.0   
              2    CHI     0.000000     0.000000        0.000000         0.0   
              3    SEA     0.000000     0.000000        0.000000         0.0   
              4    NO      3.333333    12.666667        4.000000         0.0   
              5    IND     3.333333     8.000000        3.333333         0.0   

                        fantasy points_ma  
Player        Week Opp                     
Aaron Rodgers 1    SF            0.000000  
              2    CHI           0.000000  
              3    SEA           0.000000  
              4    NO           14.533333  
              5    IND          16.280000

In [62]:
qb_recent_2012.tail()

comp_ma    att_ma     pct_ma  yds_ma  yds/att_ma  \
Player       Week Opp                                                      
Tyrod Taylor 4    CLE  0.000000  0.000000   0.000000     0.0    0.000000   
             5    KC   0.000000  0.000000   0.000000     0.0    0.000000   
             9    CLE  0.666667  1.000000  22.333333    10.0    3.333333   
             10   OAK  0.000000  0.000000   0.000000     0.0    0.000000   
             17   CIN  0.000000  0.333333   0.000000     0.0    0.000000   

                       td_ma  int_ma  qb rating_ma  rush_att_ma  rush_yds_ma  \
Player       Week Opp                                                          
Tyrod Taylor 4    CLE    0.0     0.0      0.000000     0.000000     0.000000   
             5    KC     0.0     0.0      0.000000     0.000000     0.000000   
             9    CLE    0.0     0.0     33.000000     0.333333     2.333333   
             10   OAK    0.0     0.0      0.000000     0.333333     1.333333   
             17   CIN    0.0     0.0     13.333333     0.333333     1.333333   

                       rush_yd/att_ma  rush_td_ma  fantasy points_ma  
Player       Week Opp                                                 
Tyrod Taylor 4    CLE        0.000000         0.0           0.000000  
             5    KC         0.000000         0.0           0.000000  
             9    CLE        2.333333         0.0           0.633333  
             10   OAK        1.333333         0.0           0.133333  
             17   CIN        1.333333         0.0           0.133333